In [1]:
import pymysql
import pandas as pd
import re
from bs4 import BeautifulSoup
from hanziconv import HanziConv
from zhon import hanzi
import string
punctuation=hanzi.punctuation + string.punctuation
punctuation = set([i for i in punctuation])
import html
conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='eXYhzAWjyvy8grwM',db='game_process',charset='utf8mb4')
sql = "SELECT id,source,game_name, user_name,comment_time,content, score,type,emotion FROM s_lcs_game_useless_clean_comments_qq WHERE source='qq_mobile'"
df = pd.read_sql(sql, conn)  # 返回的是 dataFrame 数据结构
conn.close()
df0 = df
df.head(10)

,id,source,game_name,user_name,comment_time,content,score,type,emotion
0,1,qq_mobile,&nbsp;杀神免费版,游客,1381762690,修复手机卡进行道具付费的漏洞！功能实用。速度很快。,5,2,3
1,2,qq_mobile,&nbsp;杀神免费版,游客,1382515674,不错，不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟...,5,3,3
2,3,qq_mobile,&nbsp;杀神免费版,游客,1390121576,新区，推荐。我叫一杯凉茶。一起玩,5,2,1
3,7,qq_mobile,&nbsp;杀神免费版,卿欲倾城,1411480319,人好像有点少啊，多点人就好了，还不烧钱,4,3,2
4,10,qq_mobile,100 Doors 2013,游客,1370442593,为什么42关的密码知道是多少就是不开门呢？,5,2,0
5,11,qq_mobile,100 Doors 2013,╭ァ蜡笔小新说你妖言惑众,1374533557,第二关之后多动动手机，百度有，开门方法太丰富。,4,3,3
6,13,qq_mobile,100 Doors 2013,游客,1390133919,界面简洁。界面简洁。使用方便。功能实用。界面很丑。经常卡死。闪退。,5,2,0
7,15,qq_mobile,100 Doors 2013,游客,1390747474,第四关怎么过太多广告。经常卡死。,1,3,2
8,16,qq_mobile,100 Doors 2013,游客,1390985546,可不好了，两个我都用了，可我到第二关就没路走了呢。,3,3,3
9,21,qq_mobile,100 Doors 2013,游客,1397473596,第二关过不了，烂，经常卡死。,1,3,2


In [2]:
def extract_text(x):
    soup = BeautifulSoup(x, 'html.parser')
    return soup.text

def clear_text(x):
    x = html.unescape(x)  # 反转义字符，显示真实内容。
    x = HanziConv.toSimplified(x) # 汉字繁体 转成 简体
    x = x.strip() # 去除字符 首尾的指定 字符
    return x

def remove_punctuation(x):  # 删除标点
    x = "".join([i for i in x if i not in punctuation]) # 遍历评论每个字符，若是标点，去掉，用空字符连接
    return re.sub(r'\s+', ' ', x) # 把 x 通过正则表达式 (\s 代表空格，+ 代表多个)，替换成 空字符

In [3]:
df['content_clear'] = df['content'].apply(clear_text)
df['content_clear'] = df['content_clear'].apply(remove_punctuation)
df.head()

,id,source,game_name,user_name,comment_time,content,score,type,emotion,content_clear
0,1,qq_mobile,&nbsp;杀神免费版,游客,1381762690,修复手机卡进行道具付费的漏洞！功能实用。速度很快。,5,2,3,修复手机卡进行道具付费的漏洞功能实用速度很快
1,2,qq_mobile,&nbsp;杀神免费版,游客,1382515674,不错，不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟...,5,3,3,不错不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟左...
2,3,qq_mobile,&nbsp;杀神免费版,游客,1390121576,新区，推荐。我叫一杯凉茶。一起玩,5,2,1,新区推荐我叫一杯凉茶一起玩
3,7,qq_mobile,&nbsp;杀神免费版,卿欲倾城,1411480319,人好像有点少啊，多点人就好了，还不烧钱,4,3,2,人好像有点少啊多点人就好了还不烧钱
4,10,qq_mobile,100 Doors 2013,游客,1370442593,为什么42关的密码知道是多少就是不开门呢？,5,2,0,为什么42关的密码知道是多少就是不开门呢


In [4]:
import jieba
def jieba_word(x):
    return [ ''.join(i) for i in  jieba.cut(x)]  # 默认是精确模式

In [5]:
df['jieba_word'] = df['content_clear'].apply(jieba_word)
df['jieba_word']

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.033 seconds.
Prefix dict has been built succesfully.


0              [修复, 手机卡, 进行, 道具, 付费, 的, 漏洞, 功能, 实用, 速度, 很快]
1         [不错, 不错, 这个, 游戏, 非常, 好玩, 随然, 登陆, 时间, 比较, 长, 但,...
2                             [新区, 推荐, 我, 叫, 一杯, 凉茶, 一起, 玩]
3               [人, 好像, 有点, 少, 啊, 多点, 人, 就, 好, 了, 还, 不, 烧钱]
4              [为什么, 42, 关, 的, 密码, 知道, 是, 多少, 就是, 不, 开门, 呢]
5              [第二, 关, 之后, 多, 动动手, 机, 百度, 有, 开门, 方法, 太, 丰富]
6         [界面, 简洁, 界面, 简洁, 使用方便, 功能, 实用, 界面, 很丑, 经常, 卡死,...
7                              [第四, 关, 怎么, 过太多, 广告, 经常, 卡死]
8         [可, 不好, 了, 两个, 我, 都, 用, 了, 可, 我, 到, 第二, 关就, 没路...
9                                 [第二, 关, 过, 不了, 烂, 经常, 卡死]
10                               [为, 毛, 35, 对, 了, 也, 没, 反应]
11                              [为什么, 36, 关对, 了, 门, 打不开, 啊]
12                  [第十, 关, 怎么, 过, 啊, 速度, 很快, 界面, 很丑, 使用方便]
13        [你们, 大多, 停留, 在, 25, 关, 那, 左右, 了, 我, 自己, 到, 40,...
14                                      [什么, 玩意, 啊, 垃圾, 闪退]
15                                [二十五, 关, 开门, 了, 为什么, 进不去]
16                               [不错, 使用

In [6]:
# 词性标注
# from jieba import posseg
# pos = list(posseg.lcut("我非常爱文本数据分析"))
# for a,b in pos:
#     print(a)
#     print(b)

In [7]:
# 生成词典
from gensim import corpora
words = [x for x in df['jieba_word']]
d = corpora.Dictionary(words)
for x,y in d.items():
    print(x,":", y)
# d是词典，有55065个词，可以删除一些 



0 : 付费
1 : 修复
2 : 功能
3 : 实用
4 : 很快
5 : 手机卡
6 : 漏洞
7 : 的
8 : 进行
9 : 速度
10 : 道具
11 : 一下
12 : 一两分钟
13 : 一起
14 : 不错
15 : 也
16 : 了
17 : 但
18 : 吧
19 : 大家
20 : 好玩
21 : 左右
22 : 差不多
23 : 我
24 : 时间
25 : 是
26 : 来
27 : 欢迎
28 : 比较
29 : 游戏
30 : 玩
31 : 登陆
32 : 看
33 : 跟
34 : 这个
35 : 长
36 : 随然
37 : 非常
38 : 一杯
39 : 凉茶
40 : 叫
41 : 推荐
42 : 新区
43 : 不
44 : 人
45 : 啊
46 : 多点
47 : 好
48 : 好像
49 : 少
50 : 就
51 : 有点
52 : 烧钱
53 : 还
54 : 42
55 : 为什么
56 : 关
57 : 呢
58 : 多少
59 : 密码
60 : 就是
61 : 开门
62 : 知道
63 : 丰富
64 : 之后
65 : 动动手
66 : 多
67 : 太
68 : 方法
69 : 有
70 : 机
71 : 百度
72 : 第二
73 : 使用方便
74 : 卡死
75 : 很丑
76 : 界面
77 : 简洁
78 : 经常
79 : 闪退
80 : 广告
81 : 怎么
82 : 第四
83 : 过太多
84 : 不好
85 : 两个
86 : 关就
87 : 到
88 : 可
89 : 没路
90 : 用
91 : 走
92 : 都
93 : 不了
94 : 烂
95 : 过
96 : 35
97 : 为
98 : 反应
99 : 对
100 : 毛
101 : 没
102 : 36
103 : 关对
104 : 打不开
105 : 门
106 : 第十
107 : 25
108 : 40
109 : 不行
110 : 你们
111 : 停留
112 : 前
113 : 在
114 : 大多
115 : 就行了
116 : 思考
117 : 最
118 : 机关
119 : 简单
120 : 自己
121 : 说实话
122 : 那
123 : 静静的
124 : 什么
125 : 垃圾
126 :

1813 : 一大
1814 : 变
1815 : 口袋
1816 : 圈
1817 : 消失
1818 : 相当
1819 : 高端
1820 : 鱼
1821 : 穿
1822 : 起码
1823 : 这家
1824 : 防弹衣
1825 : 3d
1826 : 半个
1827 : 好纯
1828 : 月
1829 : 爽爆
1830 : 质感
1831 : 不管
1832 : 付出
1833 : 可玩度
1834 : 小小
1835 : 度
1836 : 是从
1837 : 来看
1838 : 爆类
1839 : 等等
1840 : 认为
1841 : 专打
1842 : 五六十万
1843 : 圆盘
1844 : 晚上
1845 : 有卡分
1846 : 没白
1847 : 爆炸
1848 : 整体
1849 : 相比
1850 : 粉
1851 : 许多
1852 : 真正
1853 : 超棒
1854 : 超炫
1855 : 主要
1856 : 家
1857 : 游戏币
1858 : 达人
1859 : 3D
1860 : 一类
1861 : 儿时
1862 : 回忆
1863 : 城
1864 : 电玩
1865 : 非常少
1866 : 杀分
1867 : 特权
1868 : 坏
1869 : 客服
1870 : 问
1871 : 千炮
1872 : 反正
1873 : 挂
1874 : 更加
1875 : 有万炮
1876 : 房
1877 : 比赢
1878 : 轮
1879 : 守
1880 : 太累
1881 : 棋牌
1882 : 游玩
1883 : 玩网
1884 : 选择
1885 : ffc579
1886 : 二万
1887 : 伽
1888 : 吓
1889 : 微
1890 : 易懂
1891 : 珂
1892 : 腿现
1893 : 芬腿现
1894 : yu68580
1895 : ☔
1896 : 伽薇
1897 : 芬腿线
1898 : bcc692
1899 : 三十
1900 : 华丽
1901 : 吓芬
1902 : 嘉味
1903 : 柯上
1904 : 概率
1905 : 美观
1906 : 褪线
1907 : 身边
1908 : BCD266
1909 : 伽味
1910 : 操
1911 : 独特
1912

2981 : 绕下
2982 : 跳楼
2983 : 还给
2984 : 2B
2985 : 创
2986 : 别说
2987 : 货
2988 : 大冒险
2989 : 滑雪
2990 : 走过
2991 : 路过
2992 : 坑人
2993 : 明摆着
2994 : 诱惑
2995 : 他妹
2996 : 弄出来
2997 : 鄙视
2998 : 会弹
2999 : 流畅性
3000 : 顺畅
3001 : 付钱
3002 : 刚跑
3003 : 飞行
3004 : k8k6km
3005 : 尚
3006 : 带
3007 : 拁
3008 : 致富
3009 : 苛
3010 : 褪
3011 : 走向
3012 : 带劲
3013 : 720
3014 : 分读
3015 : 北大
3016 : 高考
3017 : 不玩
3018 : 拿来
3019 : 擦特
3020 : 早
3021 : 两车
3022 : 关太
3023 : 惊险
3024 : 磨蹭
3025 : 聪明
3026 : 94
3027 : 小孩子
3028 : 扣掉
3029 : 来个
3030 : 斗
3031 : 四不像
3032 : 飙车
3033 : 扣费特
3034 : 坑坑
3035 : 不酉要
3036 : 一会儿
3037 : 玩一关
3038 : 游戏猪
3039 : 上当
3040 : 刷上去
3041 : 玩不信
3042 : 评分
3043 : 跳出
3044 : 个币
3045 : 马拉
3046 : 反义词
3047 : 星太垃
3048 : 尼
3049 : 枪
3050 : 妈操
3051 : 蛋子
3052 : 压力
3053 : 昨天晚上
3054 : 炫酷
3055 : 41
3056 : 两辆车
3057 : 你好
3058 : 八天
3059 : 观
3060 : 野马
3061 : Szsy
3062 : Ι
3063 : п
3064 : с
3065 : 涩
3066 : 片
3067 : 玩累
3068 : 改改
3069 : 方式
3070 : 车升
3071 : 取
3072 : 活动
3073 : 部
3074 : 副
3075 : 圈里
3076 : 楼
3077 : 想要
3078 : 辆
3079 : 刷刷
3080 : 

3981 : 心脏
3982 : 紧张
3983 : 卡玩得
3984 : 偶也来
3985 : 凑热闹
3986 : 当钱花
3987 : 耳目一新
3988 : 凑合
3989 : 影子
3990 : 偶爽
3991 : 劲
3992 : 爽呆了
3993 : 回味无穷
3994 : 脑子里
3995 : 误
3996 : 孤芳自赏
3997 : 爽歪歪
3998 : 手快
3999 : 挺美挺
4000 : 光玩
4001 : 劲儿
4002 : 再烂
4003 : 不赞
4004 : 挺棒
4005 : 透美
4006 : 玩不烦
4007 : 十元
4008 : 慎重考虑
4009 : 倒闭
4010 : 畜牲
4011 : 钱愿
4012 : 不解
4013 : 一身
4014 : 教刷
4015 : 钻保
4016 : 顶级
4017 : 卡都关
4018 : 尽量
4019 : 无线网络
4020 : 没扣
4021 : H22B22
4022 : 广大
4023 : 淋漓
4024 : 玩能
4025 : 不出
4026 : 完人
4027 : 人会卡
4028 : 挺萌
4029 : 吓哭
4030 : 口味
4031 : 合胃口
4032 : 重
4033 : 出人命
4034 : 密集
4035 : 恐惧症
4036 : 羊驼
4037 : 压倒
4038 : 所
4039 : 打败
4040 : 看成
4041 : 精髓
4042 : 羊
4043 : 驼
4044 : ฅ
4045 : ﻌ
4046 : 单调
4047 : 比一
4048 : 进化
4049 : N
4050 : 世
4051 : 哪咤
4052 : 多头
4053 : 好狠
4054 : 辛苦
4055 : 嘤
4056 : 跳台
4057 : 跳过
4058 : 时玩
4059 : 清新
4060 : 上古
4061 : 助手
4062 : 开服
4063 : 方便快捷
4064 : 礼包
4065 : 领取
4066 : tm
4067 : 包装
4068 : 迄今
4069 : 分等级
4070 : 是否
4071 : 程度
4072 : 广
4073 : 提交
4074 : 要饭
4075 : 认钱
4076 : 赏
4077 : 跑不动
4078 : 闪卸
4

5060 : 太丑
5061 : 一千
5062 : 滑稽
5063 : 拉机
5064 : boss
5065 : 外表
5066 : 当射术
5067 : 翅膀
5068 : 英文名
5069 : 蟹
5070 : 角色扮演
5071 : 那时
5072 : 玩不过来
5073 : 大陆
5074 : 或
5075 : 歌曲
5076 : 港台
5077 : 拇指
5078 : 市场
5079 : 果然如此
5080 : 不造
5081 : 干毛滴
5082 : 日文
5083 : dota
5084 : 5v5
5085 : 创建
5086 : 高度
5087 : 宝贝
5088 : 扎
5089 : 熊
5090 : 一系列
5091 : 熊猫
5092 : 它们
5093 : 来接
5094 : 风景画
5095 : 坐龙
5096 : 棵
5097 : 身上
5098 : 连机
5099 : 东西连
5100 : 肉
5101 : 农场
5102 : 房子
5103 : 太乱
5104 : 红鞋子
5105 : 不认
5106 : 爱上
5107 : 货币
5108 : 需求
5109 : gvhvhgj
5110 : 燥
5111 : 有鬼
5112 : 决斗
5113 : fantastic
5114 : 两下
5115 : 49
5116 : 级
5117 : 完玩
5118 : 很帅
5119 : 多一点
5120 : 地域
5121 : 广告牌
5122 : 多死
5123 : 有负
5124 : 我点
5125 : 一邦邦
5126 : 几十秒
5127 : 我昂
5128 : 私聊
5129 : 空余
5130 : 不当
5131 : 当讲
5132 : emmm
5133 : 攀爬
5134 : 敬请期待
5135 : 森林
5136 : 二十分钟
5137 : 无奈
5138 : 缺陷
5139 : 测试
5140 : 一跑
5141 : 对手
5142 : 我勒个
5143 : 摔
5144 : 说真的
5145 : 到此一游
5146 : 小六
5147 : Duang
5148 : 注意力
5149 : 26
5150 : 关玩不出
5151 : 188888
5152 : 弹扣
5153 : 疯
5154 : 巴
5155 : 

...

50153 : 焉
50154 : 礼义
50155 : 得癌
50156 : 残去
50157 : 绝种
50158 : 惊无喜
50159 : 感慨万千
50160 : 书包
50161 : 往家
50162 : 搬个
50163 : 铃声响起
50164 : 几星期
50165 : 四娃
50166 : 一不太坑
50167 : 武斗
50168 : 扣赞
50169 : 有折
50170 : 服游
50171 : 玩私
50172 : 前进方向
50173 : 投放
50174 : 提前量
50175 : 鱼雷
50176 : 货滚
50177 : 搁
50178 : 傻逼乱
50179 : 干扣
50180 : 货辣鸡
50181 : AAP
50182 : 句子
50183 : 六娃
50184 : 器灵大娃
50185 : 戎马
50186 : 空儒
50187 : 挥刀自宫
50188 : 欲练
50189 : 个大鸡
50190 : 关团
50191 : 最右观
50192 : 配角
50193 : 2200
50194 : 更多信
50195 : 求点
50196 : 胜出
50197 : 钻否
50198 : 极快
50199 : 声调
50200 : 294
50201 : 会拉
50202 : 提成
50203 : 抄尼魅
50204 : wohaimeiyowqn
50205 : 粗来
50206 : 但论
50207 : 纸上
50208 : Cjhfcvu
50209 : 党赞
50210 : 杂太强
50211 : 易安
50212 : 社而
50213 : 打起精神
50214 : 蔬菜水果
50215 : 厚实
50216 : kid
50217 : 行买
50218 : ೃ
50219 : 星请
50220 : illusions
50221 : 吗啡
50222 : 止咳
50223 : 泄密
50224 : 蛔虫
50225 : 行货
50226 : 邓亚萍
50227 : 下旋
50228 : 一星烂
50229 : 角落
50230 : anwser
50231 : 合唱团
50232 : 吉它
50233 : 抽搐起来
50234 : 地儿
50235 : 连读
50236 : LAJI
50237 : 破和特
50

51318 : 云中
51319 : 是丽颖演
51320 : 玉观音
51321 : 换下来
51322 : 守备军
51323 : 流动
51324 : 软次
51325 : 3733
51326 : 盗贼
51327 : 行者
51328 : 不卡有
51329 : 多得花
51330 : 平直
51331 : 眼急手快
51332 : 超杀
51333 : 600M
51334 : 再玩过
51335 : 加长
51336 : 一击必杀
51337 : 休眠
51338 : 车虐
51339 : 减在
51340 : 奖池
51341 : 审查员
51342 : 打太开
51343 : 因岷叔来
51344 : 粉们
51345 : 赞岷
51346 : 要教
51347 : 大猩猩
51348 : 不大点
51349 : 使点
51350 : 买橙长
51351 : 搬出来
51352 : 超神迷
51353 : 会露
51354 : 彦
51355 : 淋
51356 : 葛小伦
51357 : 驼背
51358 : 神学院
51359 : 追超
51360 : 自动记录
51361 : 杂七乱
51362 : 以北
51363 : 很僵
51364 : 白装
51365 : 慰护
51366 : 好放
51367 : 托祝
51368 : 战卡
51369 : 远度
51370 : 发行者
51371 : 哈尼
51372 : 海天
51373 : 整不起
51374 : 没骂
51375 : 狗配
51376 : 充年
51377 : 服务器端
51378 : 点区
51379 : 第俩
51380 : sky
51381 : xiao8
51382 : 拉取
51383 : 妳们
51384 : 核审
51385 : Vip68
51386 : vip7
51387 : 爆比
51388 : 整一
51389 : 不详
51390 : 伦德
51391 : 冰盖
51392 : 提莫寒
51393 : 下九游
51394 : 旧区
51395 : 刀塔渣
51396 : 慌进
51397 : 货赴
51398 : 需然
51399 : 办公
51400 : 电钻
51401 : 牛得
51402 : 泼墨
51403 : 步赢
51404 : 假不

52481 : 紧张感
52482 : 地板砖
52483 : 中短途
52484 : 护身符
52485 : 左下会
52486 : 跑掉
52487 : 0917
52488 : 104104
52489 : 有幅
52490 : 洗发液
52491 : 浴池
52492 : 老板娘
52493 : 那层
52494 : 溶液
52495 : 纯铛
52496 : 压看
52497 : 一震
52498 : 虎躯
52499 : 出让
52500 : 闹鬼
52501 : 怕鬼
52502 : 挺怕
52503 : 进真滴
52504 : 檫
52505 : 好怕
52506 : 安慰
52507 : 下出
52508 : 二下
52509 : 太可伯
52510 : 走会
52511 : 退着
52512 : 马上转
52513 : 鬼会
52514 : 屏吓
52515 : 电筒
52516 : 地鬼
52517 : 精神科
52518 : 神出鬼没
52519 : 这鬼
52520 : 头皮发麻
52521 : 拉开距离
52522 : 勿近
52523 : 胆小鬼
52524 : 傻傻的
52525 : 张开
52526 : Pretty
52527 : 鬼片
52528 : 这鬼长
52529 : 三头六臂
52530 : 呵护
52531 : 拼了命
52532 : 真燥
52533 : 宁可
52534 : 糟榻
52535 : 诺曼底
52536 : 家中
52537 : 差史
52538 : 我靠点
52539 : 1112131415
52540 : 12345
52541 : 排对
52542 : 道门
52543 : 不玩太
52544 : 这太好
52545 : 关毕
52546 : illegal
52547 : 字烂进
52548 : 进不击
52549 : 相关联
52550 : 心给
52551 : 哪才
52552 : 解题
52553 : 退太气
52554 : 搞乱
52555 : 没电是
52556 : 杀器
52557 : 妈能
52558 : 超爱赞
52559 : 莫有
52560 : 抢用
52561 : 拆开
52562 : 火药
52563 : 白箱
52564 : 酒柜
52565 : 雷管
52566 :

53472 : 2550733549
53473 : 叩
53474 : 大举
53475 : 非花
53476 : 卡在币
53477 : 上个星期
53478 : 两块钱
53479 : 防盗
53480 : 合咯
53481 : 敷
53482 : 萤草
53483 : 御魂
53484 : ssR
53485 : 慢升星
53486 : 陆果
53487 : VIP11
53488 : The
53489 : ″
53490 : 说实
53491 : 饕餮
53492 : 移过去
53493 : Eminem
53494 : 后空翻
53495 : 过沟
53496 : inside
53497 : 漫漫长夜
53498 : 进爱
53499 : 一关点
53500 : 元条
53501 : 热是
53502 : 手游满
53503 : 硬刚
53504 : 比列王
53505 : 龙会
53506 : 别玩下
53507 : 外交官
53508 : 金主
53509 : 隆之王
53510 : 单靠
53511 : 洗劫
53512 : 谋士
53513 : 普了
53514 : 存储卡
53515 : 想弃
53516 : 堡升
53517 : 起兵
53518 : 黑吞
53519 : 挺萌萌
53520 : 玩六胡
53521 : 细则
53522 : 凑不起
53523 : 跟家
53524 : 当面
53525 : 有话
53526 : 阿闪
53527 : 回老家
53528 : 个人资料
53529 : 四核机
53530 : 质控
53531 : 上忙
53532 : 油腻感
53533 : 优缺点
53534 : 恩玩
53535 : 这俩
53536 : 评关
53537 : 十胡起
53538 : 胡起
53539 : 湖北
53540 : 赢输
53541 : 岳母
53542 : 岳父
53543 : 拥护者
53544 : つ
53545 : 剥夺
53546 : 迫使
53547 : 黄金岛
53548 : 垃吸
53549 : 战网
53550 : 列出来
53551 : 差暴
53552 : 标清外
53553 : 用星灵
53554 : 地图分析
53555 : facebook
53556 : 白才
53557 : 老

54574 : caonim
54575 : 痹
54576 : 贵麻
54577 : 算区
54578 : 三米
54579 : 我百玩
54580 : 脚步声
54581 : 行用
54582 : 笨拙
54583 : keng
54584 : 党咯
54585 : 虽能
54586 : 502
54587 : 妄想
54588 : 退刷成
54589 : iPhone4
54590 : Wower
54591 : 野队
54592 : 药剂
54593 : 阔叶
54594 : ADD
54595 : 表示遗憾
54596 : 无处不在
54597 : 气墙
54598 : 冲领
54599 : 账首
54600 : qvq
54601 : 魔龙
54602 : 每一项
54603 : 百十块
54604 : 送小冰龙
54605 : 一比进
54606 : 叫迪丽
54607 : 延时
54608 : 制毁
54609 : 定价
54610 : WOW
54611 : 名南溟
54612 : 高塔
54613 : FD
54614 : 曜
54615 : 水木
54616 : 罗瑟兰
54617 : 龙骨
54618 : 快放
54619 : 沙虫
54620 : 充钱力
54621 : 硬直
54622 : 打遍
54623 : 多人乱战
54624 : 受限
54625 : 乳猪
54626 : 金党
54627 : 552841469
54628 : 来者
54629 : 钱算
54630 : 卡拔
54631 : 逼玩应
54632 : 难获
54633 : 拉色
54634 : mei
54635 : 血命
54636 : 读图
54637 : 面目全非
54638 : 1019
54639 : 多气
54640 : 中性
54641 : 分长
54642 : 一卡中
54643 : 飞禽走兽
54644 : TTD166
54645 : 乘三得
54646 : 八灯
54647 : 拖子
54648 : 最黑
54649 : 机全
54650 : 785234393
54651 : 众发
54652 : 周入
54653 : 588
54654 : 去弥玛
54655 : 弥玛
54656 : VIP1516
54657 : h22h28
546

In [8]:
# dict 的一系列了属性(非简单的字典)： 1对应字典里序号为1的词， 284 在x个文档中出现过
for x,y in d.dfs.items():
    print(x,":",y)

1 : 284
5 : 70
8 : 99
10 : 316
0 : 176
7 : 62883
6 : 80
2 : 1687
3 : 1311
9 : 1346
4 : 1228
14 : 17240
34 : 11600
29 : 47265
37 : 9265
20 : 23061
36 : 5
31 : 932
24 : 3330
28 : 1627
35 : 202
17 : 2122
23 : 29413
32 : 2354
16 : 47427
11 : 3159
15 : 12385
22 : 401
25 : 18784
12 : 4
21 : 129
18 : 5095
27 : 188
19 : 6131
26 : 4014
33 : 1687
13 : 1898
30 : 28300
42 : 580
41 : 3480
40 : 836
38 : 1
39 : 1
44 : 4636
48 : 513
51 : 2677
49 : 1136
45 : 10566
46 : 333
50 : 19832
47 : 20062
53 : 14533
43 : 14437
52 : 122
55 : 4547
54 : 29
56 : 1094
59 : 188
62 : 3078
58 : 505
60 : 9937
61 : 16
57 : 3305
72 : 305
64 : 1108
66 : 6588
65 : 3
70 : 57
71 : 117
69 : 10795
68 : 134
67 : 7024
63 : 802
76 : 2238
77 : 1030
73 : 789
75 : 329
78 : 1170
74 : 1036
79 : 4222
82 : 109
81 : 2059
83 : 1
80 : 4413
88 : 776
84 : 1731
85 : 558
92 : 22930
90 : 3906
87 : 4389
86 : 125
89 : 2
91 : 409
95 : 2614
93 : 4866
94 : 689
97 : 669
100 : 395
96 : 31
99 : 1041
101 : 5767
98 : 483
102 : 38
103 : 1
105 : 64
104 : 2893

880 : 169
879 : 85
878 : 18
881 : 2
882 : 1
884 : 200
883 : 57
885 : 135
886 : 216
887 : 227
891 : 125
888 : 148
889 : 263
890 : 5316
895 : 535
893 : 1598
894 : 278
897 : 1159
896 : 50
892 : 14
899 : 86
898 : 89
902 : 5
900 : 14
901 : 20
903 : 1390
904 : 247
908 : 245
907 : 905
906 : 474
905 : 383
912 : 596
913 : 114
910 : 180
911 : 305
909 : 180
914 : 347
917 : 68
918 : 745
916 : 154
915 : 163
923 : 1
922 : 12
921 : 2
920 : 2
924 : 5
919 : 514
925 : 79
927 : 273
926 : 1020
928 : 61
929 : 74
930 : 2185
931 : 6
932 : 1511
933 : 688
934 : 319
935 : 23
936 : 381
937 : 17
939 : 1487
938 : 56
940 : 391
941 : 16
942 : 51
943 : 1210
944 : 18
947 : 17
945 : 55
946 : 1
948 : 729
950 : 344
949 : 11
954 : 218
956 : 69
951 : 1
952 : 60
953 : 199
955 : 8
959 : 1456
961 : 216
960 : 92
957 : 387
958 : 1110
962 : 92
963 : 11
966 : 293
968 : 307
964 : 80
967 : 48
965 : 51
969 : 185
977 : 92
972 : 280
971 : 198
975 : 5
970 : 16
973 : 31
974 : 129
976 : 81
979 : 334
978 : 455
980 : 118
981 : 268
987 : 41

1898 : 67
1899 : 64
1911 : 204
1910 : 395
1912 : 230
1909 : 13
1908 : 32
1916 : 62
1915 : 125
1918 : 122
1914 : 218
1920 : 46
1921 : 27
1917 : 78
1913 : 2
1919 : 580
1922 : 960
1927 : 60
1924 : 1
1926 : 102
1923 : 29
1925 : 16
1931 : 4
1930 : 19
1929 : 2
1933 : 30
1928 : 30
1932 : 19
1934 : 273
1937 : 242
1936 : 357
1938 : 102
1940 : 164
1939 : 85
1935 : 77
1942 : 1
1943 : 1
1944 : 4
1945 : 196
1941 : 49
1946 : 108
1947 : 2
1952 : 160
1951 : 150
1950 : 202
1948 : 450
1949 : 29
1959 : 2
1956 : 153
1953 : 3
1958 : 1
1957 : 216
1954 : 1
1955 : 1
1960 : 54
1961 : 18
1963 : 24
1964 : 234
1965 : 48
1962 : 5
1969 : 129
1967 : 602
1968 : 198
1971 : 62
1970 : 83
1966 : 11
1972 : 7
1973 : 414
1974 : 1
1975 : 2
1977 : 235
1976 : 32
1978 : 760
1979 : 26
1980 : 341
1985 : 89
1983 : 31
1982 : 40
1981 : 82
1987 : 782
1986 : 82
1984 : 217
1988 : 111
1991 : 332
1990 : 35
1989 : 1
1992 : 8
1993 : 4
1994 : 18
1995 : 9
1997 : 49
1996 : 130
1998 : 740
2000 : 1
1999 : 45
2001 : 248
2002 : 421
2005 : 69
2004

2738 : 17
2737 : 4
2740 : 101
2745 : 47
2747 : 1
2739 : 75
2746 : 28
2741 : 56
2742 : 16
2748 : 25
2743 : 20
2744 : 3
2749 : 16
2756 : 12
2751 : 1
2757 : 1
2755 : 11
2752 : 1
2750 : 86
2754 : 11
2753 : 19
2758 : 30
2760 : 12
2761 : 2
2759 : 431
2762 : 84
2763 : 80
2764 : 1
2766 : 107
2765 : 6
2767 : 85
2768 : 5
2769 : 36
2770 : 336
2772 : 270
2774 : 35
2773 : 8
2771 : 183
2778 : 13
2775 : 104
2777 : 2
2776 : 1
2779 : 11
2781 : 62
2780 : 339
2782 : 40
2784 : 112
2783 : 294
2785 : 1
2787 : 17
2786 : 18
2788 : 3
2791 : 8
2792 : 181
2789 : 1
2790 : 12
2795 : 6
2793 : 18
2794 : 132
2796 : 1
2798 : 32
2797 : 148
2799 : 20
2800 : 21
2801 : 143
2802 : 121
2803 : 247
2804 : 12
2805 : 96
2806 : 10
2807 : 3
2808 : 131
2811 : 809
2810 : 87
2809 : 127
2813 : 29
2814 : 12
2812 : 14
2825 : 31
2822 : 12
2816 : 2
2821 : 15
2815 : 5
2824 : 26
2819 : 12
2823 : 2
2828 : 11
2818 : 46
2827 : 3
2826 : 13
2820 : 11
2817 : 15
2829 : 1
2830 : 10
2831 : 7
2832 : 33
2834 : 3
2833 : 1
2835 : 57
2836 : 1
2838 : 1
2

3896 : 18
3895 : 66
3893 : 118
3897 : 2
3898 : 1
3899 : 2
3900 : 1
3903 : 73
3901 : 5
3902 : 70
3904 : 26
3905 : 1
3906 : 1
3913 : 4
3907 : 134
3915 : 55
3912 : 14
3914 : 27
3911 : 28
3908 : 28
3909 : 23
3910 : 28
3916 : 29
3917 : 1
3918 : 21
3920 : 177
3919 : 6
3921 : 270
3922 : 1
3923 : 36
3925 : 22
3924 : 432
3926 : 3
3927 : 82
3928 : 92
3929 : 28
3930 : 1
3931 : 1
3932 : 1
3933 : 3
3934 : 15
3935 : 4
3936 : 1
3937 : 89
3938 : 910
3939 : 7
3940 : 28
3942 : 27
3941 : 39
3943 : 13
3944 : 282
3945 : 1
3946 : 9
3947 : 7
3948 : 101
3949 : 26
3950 : 18
3951 : 44
3952 : 4
3953 : 140
3955 : 8
3954 : 25
3956 : 1
3957 : 64
3958 : 1
3959 : 104
3960 : 2
3961 : 3
3962 : 16
3964 : 52
3963 : 1
3965 : 3
3966 : 70
3967 : 105
3968 : 1
3969 : 66
3971 : 30
3970 : 70
3972 : 7
3973 : 159
3974 : 74
3975 : 26
3976 : 354
3977 : 1
3978 : 66
3979 : 1
3980 : 1
3981 : 17
3982 : 43
3983 : 1
3984 : 1
3985 : 7
3986 : 1
3987 : 8
3988 : 44
3989 : 12
3990 : 1
3992 : 8
3991 : 41
3993 : 12
3994 : 8
3995 : 7
3996 : 1
39

...

50565 : 1
50564 : 1
50566 : 1
50567 : 1
50568 : 1
50569 : 1
50570 : 1
50571 : 2
50572 : 1
50573 : 1
50574 : 1
50575 : 1
50576 : 1
50577 : 1
50578 : 1
50580 : 1
50579 : 1
50581 : 1
50582 : 1
50583 : 1
50585 : 1
50584 : 1
50586 : 1
50587 : 1
50588 : 1
50589 : 1
50590 : 1
50591 : 1
50593 : 1
50592 : 1
50594 : 1
50595 : 1
50597 : 1
50599 : 1
50598 : 1
50596 : 1
50600 : 1
50601 : 1
50602 : 1
50603 : 1
50604 : 1
50605 : 1
50606 : 1
50607 : 1
50608 : 1
50609 : 1
50610 : 1
50611 : 1
50612 : 1
50613 : 1
50614 : 1
50615 : 1
50616 : 1
50617 : 1
50619 : 1
50618 : 2
50620 : 1
50621 : 1
50622 : 2
50623 : 1
50624 : 1
50626 : 1
50628 : 1
50625 : 1
50627 : 1
50629 : 1
50630 : 1
50632 : 1
50631 : 1
50637 : 1
50636 : 1
50633 : 1
50635 : 1
50634 : 1
50638 : 1
50639 : 1
50640 : 1
50641 : 1
50643 : 1
50642 : 1
50644 : 1
50645 : 1
50646 : 1
50647 : 1
50648 : 1
50649 : 1
50650 : 1
50651 : 1
50652 : 1
50653 : 1
50654 : 1
50655 : 1
50656 : 1
50657 : 1
50658 : 1
50659 : 1
50661 : 1
50660 : 1
50662 : 1
50663 : 1


51730 : 1
51731 : 1
51732 : 1
51733 : 1
51734 : 1
51735 : 1
51736 : 1
51737 : 1
51738 : 2
51739 : 1
51740 : 1
51741 : 2
51742 : 3
51743 : 1
51744 : 1
51747 : 2
51746 : 1
51748 : 1
51749 : 1
51745 : 1
51750 : 1
51751 : 1
51752 : 1
51753 : 1
51754 : 2
51755 : 1
51756 : 1
51757 : 1
51758 : 3
51759 : 1
51760 : 1
51761 : 1
51762 : 1
51763 : 1
51764 : 1
51765 : 1
51766 : 1
51770 : 1
51767 : 3
51769 : 2
51768 : 1
51772 : 1
51771 : 2
51773 : 1
51774 : 1
51775 : 1
51776 : 1
51777 : 1
51778 : 1
51779 : 1
51780 : 1
51781 : 1
51782 : 1
51784 : 2
51783 : 1
51785 : 1
51786 : 1
51787 : 1
51788 : 1
51789 : 1
51790 : 1
51791 : 1
51793 : 1
51792 : 1
51794 : 1
51795 : 1
51796 : 1
51797 : 1
51798 : 1
51799 : 1
51800 : 1
51801 : 1
51802 : 1
51803 : 1
51804 : 2
51805 : 1
51806 : 2
51807 : 1
51809 : 1
51810 : 1
51808 : 1
51811 : 1
51813 : 1
51812 : 1
51814 : 1
51815 : 1
51816 : 1
51817 : 1
51818 : 1
51819 : 1
51820 : 1
51821 : 1
51822 : 1
51823 : 1
51824 : 1
51825 : 1
51826 : 1
51827 : 1
51828 : 1
51829 : 1


52900 : 1
52901 : 1
52903 : 1
52902 : 1
52904 : 1
52905 : 1
52906 : 1
52907 : 1
52908 : 1
52909 : 1
52910 : 1
52912 : 1
52911 : 1
52913 : 1
52914 : 1
52915 : 1
52917 : 1
52916 : 1
52918 : 1
52920 : 1
52921 : 1
52919 : 1
52922 : 1
52923 : 1
52925 : 1
52924 : 1
52926 : 1
52927 : 1
52928 : 1
52929 : 1
52930 : 1
52931 : 1
52932 : 1
52933 : 1
52934 : 1
52935 : 1
52936 : 1
52939 : 1
52938 : 1
52937 : 1
52941 : 1
52940 : 1
52942 : 1
52943 : 1
52945 : 1
52944 : 1
52946 : 1
52947 : 1
52948 : 1
52949 : 1
52950 : 1
52951 : 1
52952 : 1
52953 : 1
52954 : 1
52955 : 1
52956 : 1
52957 : 1
52958 : 1
52959 : 1
52960 : 2
52961 : 1
52962 : 1
52967 : 1
52964 : 1
52966 : 1
52963 : 1
52965 : 1
52968 : 2
52969 : 1
52971 : 1
52972 : 3
52970 : 1
52974 : 1
52973 : 4
52975 : 1
52976 : 1
52977 : 1
52978 : 1
52979 : 1
52982 : 1
52980 : 1
52981 : 1
52984 : 1
52988 : 1
52986 : 1
52989 : 1
52987 : 1
52983 : 1
52985 : 1
52990 : 1
52991 : 1
52992 : 1
52994 : 1
52993 : 1
52995 : 1
52996 : 1
52997 : 1
52999 : 1
52998 : 1


53894 : 2
53895 : 1
53896 : 1
53897 : 1
53898 : 1
53901 : 1
53900 : 1
53899 : 1
53902 : 1
53903 : 1
53904 : 1
53905 : 1
53906 : 1
53907 : 1
53910 : 1
53909 : 1
53908 : 1
53911 : 1
53912 : 1
53913 : 1
53914 : 1
53915 : 1
53916 : 1
53917 : 1
53918 : 1
53919 : 1
53920 : 1
53922 : 1
53921 : 1
53923 : 1
53924 : 2
53926 : 1
53927 : 1
53925 : 1
53928 : 1
53929 : 1
53930 : 1
53931 : 1
53932 : 1
53933 : 1
53934 : 1
53935 : 1
53936 : 1
53937 : 1
53938 : 1
53940 : 2
53939 : 1
53941 : 1
53942 : 1
53943 : 1
53944 : 1
53945 : 1
53946 : 1
53947 : 1
53948 : 2
53949 : 1
53951 : 1
53950 : 1
53952 : 1
53953 : 1
53954 : 1
53955 : 1
53956 : 1
53957 : 1
53958 : 1
53959 : 1
53961 : 1
53962 : 1
53960 : 1
53963 : 2
53964 : 1
53965 : 1
53966 : 1
53967 : 1
53968 : 1
53970 : 1
53971 : 1
53969 : 1
53972 : 1
53973 : 1
53974 : 1
53975 : 1
53976 : 2
53977 : 1
53978 : 1
53979 : 1
53980 : 1
53981 : 1
53982 : 1
53983 : 1
53984 : 1
53985 : 1
53986 : 1
53987 : 1
53988 : 1
53989 : 1
53990 : 1
53991 : 1
53992 : 1
53994 : 1


55046 : 1
55047 : 1
55048 : 1
55049 : 1
55050 : 1
55051 : 1
55052 : 1
55053 : 2
55054 : 1
55056 : 1
55055 : 1
55058 : 1
55057 : 1
55059 : 1
55061 : 1
55060 : 1
55062 : 1
55063 : 1
55064 : 1


In [42]:
# doc2bow  生成词频 向量， 和词频矩阵不同
word_count = [d.doc2bow(word) for word in words]
word_count # word_count [[0,1],[]....[]] 行代表文档，列代表词汇，元素为词频值



[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1)],
 [(7, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 2),
  (24, 2),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 2),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2)],
 [(13, 1), (23, 1), (30, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)],
 [(16, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1)],
 [(7, 1),
  (25, 1),
  (43, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1)],
 [(56, 1),
  (61, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1)],
 [(2, 1),
  (3, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 3),
  (77, 2),
  (78, 1),
  (79, 1)],
 [(56, 1)

In [55]:
from gensim.matutils import corpus2csc
# 构建词频 ，
# corpus2dense 稠密矩阵 csc 稀疏矩阵
corpus_matrix = corpus2csc(word_count, len(d))
a = corpus_matrix.T

######### 此时 计算两种的 夹角余弦
x = a[:10000]
similarities = cosine_similarity(x)
#print('pairwise dense output:\n {}\n'.format(similarities))
type(similarities)

numpy.ndarray

In [69]:
import numpy as np
def get_max_index(x):    
    return np.where(x == np.max(x))

TypeError: 'NoneType' object is not subscriptable

In [11]:
from gensim import corpora,models
# 计算tf-idf 值， corpus_tfidf 的列数 为该文本分词个数
tfidf_model = models.TfidfModel(word_count) 
corpus_tfidf = tfidf_model[word_count]
type(corpus_tfidf)
corpus_tfidf[0]

[(0, 0.34582593450420757),
 (1, 0.32130140733517976),
 (2, 0.22998051588948781),
 (3, 0.24290480028023728),
 (4, 0.2462569832590436),
 (5, 0.39308152340531011),
 (6, 0.38623750868734219),
 (7, 0.044527023727623814),
 (8, 0.37531563425581355),
 (9, 0.24155441043413006),
 (10, 0.31582912414499476)]

In [12]:
for x in corpus_tfidf[0]:
    print(x[0],":",x[1]) # x 是列表字元素， 是一个元组

0 : 0.345825934504
1 : 0.321301407335
2 : 0.229980515889
3 : 0.24290480028
4 : 0.246256983259
5 : 0.393081523405
6 : 0.386237508687
7 : 0.0445270237276
8 : 0.375315634256
9 : 0.241554410434
10 : 0.315829124145


In [13]:
# 转换corpus_tfidf的格式变成列数为，词典列数，这样 列对应词典的列
# corpus_matrix=corpus2csc(corpus_tfidf, len(d))
# b = corpus_matrix.T
# b.shape

In [32]:
print(corpus_tfidf)

In [15]:
# 选出每行 tfidf  》0.5 的词
def select_tfidf(y):  # y 代行元素
    global index
    tfidf_index_list = corpus_tfidf[index]
    index = index + 1
    d_index_list = [ x[0] for x in tfidf_index_list if x[1] > 0.25]
    # d字典中选出 d_index_list 代表的数
    return [d[i] for i in d_index_list]    

In [16]:
index = 0
df["tfidf_word"] = df["id"].apply(select_tfidf) # 默认为行 axis
df.head()

,id,source,game_name,user_name,comment_time,content,score,type,emotion,content_clear,jieba_word,tfidf_word
0,1,qq_mobile,&nbsp;杀神免费版,游客,1381762690,修复手机卡进行道具付费的漏洞！功能实用。速度很快。,5,2,3,修复手机卡进行道具付费的漏洞功能实用速度很快,"[修复, 手机卡, 进行, 道具, 付费, 的, 漏洞, 功能, 实用, 速度, 很快]","[付费, 修复, 手机卡, 漏洞, 进行, 道具]"
1,2,qq_mobile,&nbsp;杀神免费版,游客,1382515674,不错，不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟...,5,3,3,不错不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟左...,"[不错, 不错, 这个, 游戏, 非常, 好玩, 随然, 登陆, 时间, 比较, 长, 但,...","[一两分钟, 左右, 时间, 登陆, 随然]"
2,3,qq_mobile,&nbsp;杀神免费版,游客,1390121576,新区，推荐。我叫一杯凉茶。一起玩,5,2,1,新区推荐我叫一杯凉茶一起玩,"[新区, 推荐, 我, 叫, 一杯, 凉茶, 一起, 玩]","[一杯, 凉茶, 叫, 新区]"
3,7,qq_mobile,&nbsp;杀神免费版,卿欲倾城,1411480319,人好像有点少啊，多点人就好了，还不烧钱,4,3,2,人好像有点少啊多点人就好了还不烧钱,"[人, 好像, 有点, 少, 啊, 多点, 人, 就, 好, 了, 还, 不, 烧钱]","[人, 多点, 好像, 少, 有点, 烧钱]"
4,10,qq_mobile,100 Doors 2013,游客,1370442593,为什么42关的密码知道是多少就是不开门呢？,5,2,0,为什么42关的密码知道是多少就是不开门呢,"[为什么, 42, 关, 的, 密码, 知道, 是, 多少, 就是, 不, 开门, 呢]","[42, 关, 多少, 密码, 开门]"


In [17]:
df[["game_name","content_clear","jieba_word","tfidf_word"]].head(100)

,game_name,content_clear,jieba_word,tfidf_word
0,&nbsp;杀神免费版,修复手机卡进行道具付费的漏洞功能实用速度很快,"[修复, 手机卡, 进行, 道具, 付费, 的, 漏洞, 功能, 实用, 速度, 很快]","[付费, 修复, 手机卡, 漏洞, 进行, 道具]"
1,&nbsp;杀神免费版,不错不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟左...,"[不错, 不错, 这个, 游戏, 非常, 好玩, 随然, 登陆, 时间, 比较, 长, 但,...","[一两分钟, 左右, 时间, 登陆, 随然]"
2,&nbsp;杀神免费版,新区推荐我叫一杯凉茶一起玩,"[新区, 推荐, 我, 叫, 一杯, 凉茶, 一起, 玩]","[一杯, 凉茶, 叫, 新区]"
3,&nbsp;杀神免费版,人好像有点少啊多点人就好了还不烧钱,"[人, 好像, 有点, 少, 啊, 多点, 人, 就, 好, 了, 还, 不, 烧钱]","[人, 多点, 好像, 少, 有点, 烧钱]"
4,100 Doors 2013,为什么42关的密码知道是多少就是不开门呢,"[为什么, 42, 关, 的, 密码, 知道, 是, 多少, 就是, 不, 开门, 呢]","[42, 关, 多少, 密码, 开门]"
5,100 Doors 2013,第二关之后多动动手机百度有开门方法太丰富,"[第二, 关, 之后, 多, 动动手, 机, 百度, 有, 开门, 方法, 太, 丰富]","[开门, 动动手, 方法, 机, 百度, 第二]"
6,100 Doors 2013,界面简洁界面简洁使用方便功能实用界面很丑经常卡死闪退,"[界面, 简洁, 界面, 简洁, 使用方便, 功能, 实用, 界面, 很丑, 经常, 卡死,...","[使用方便, 很丑, 界面, 简洁]"
7,100 Doors 2013,第四关怎么过太多广告经常卡死,"[第四, 关, 怎么, 过太多, 广告, 经常, 卡死]","[关, 卡死, 经常, 第四, 过太多]"
8,100 Doors 2013,可不好了两个我都用了可我到第二关就没路走了呢,"[可, 不好, 了, 两个, 我, 都, 用, 了, 可, 我, 到, 第二, 关就, 没路...","[第二, 两个, 关就, 可, 没路, 走]"
9,100 Doors 2013,第二关过不了烂经常卡死,"[第二, 关, 过, 不了, 烂, 经常, 卡死]","[关, 第二, 卡死, 经常, 不了, 烂, 过]"


In [18]:
# import gensim
# from nltk.corpus import brown 
# model = gensim.models.Word2Vec(words, size=100, window=5, min_count=5)
# set(model.wv.vocab.keys()) # 查看模型中所有词汇

{'大师',
 '官网',
 '亮点',
 '恐惧',
 'xx',
 '高档',
 '十多种',
 '一棵',
 '好几十',
 '男孩',
 '板块',
 '断',
 '戏',
 '波动',
 '栏',
 '扣',
 '破',
 '掌上',
 '养老',
 '不惜',
 '几何',
 '好顶',
 '点半',
 '创作者',
 '领个',
 '粗暴',
 '轻轻',
 '瘾',
 '后排',
 '看花了',
 '更上一层楼',
 '舅舅',
 '要点',
 '收人',
 '斗罗',
 '茅坑',
 '敌国',
 '各位朋友',
 '一行',
 '传送',
 'ma',
 '二郎神',
 '平台',
 '啥子',
 '聪明',
 '家当',
 '说完',
 '̀',
 '真辣鸡',
 '玩人',
 '群雄',
 '冲级',
 '卡玩',
 '老王',
 'A',
 '格式',
 '没过',
 '点卡',
 '一仙',
 '群是',
 '耗电',
 '美型',
 '瓶',
 '不让',
 '圈钱',
 '上千',
 '高玩',
 '跑得快',
 '别错过',
 '酸',
 '阿',
 '地形',
 '承受',
 '坐标',
 '杀庄力',
 '终点',
 '十分钟',
 '收太多外',
 'ta',
 '范围',
 '排名',
 '多多',
 '老人家',
 '放屁',
 '打谱',
 '纸',
 '谜题',
 '德州',
 '介意',
 '赢得',
 '曹操传',
 '兵营',
 '核心',
 '代言人',
 '88',
 '太烦',
 '好多',
 '幸亏',
 '马拉',
 '不送',
 '六',
 '我网',
 '第一件',
 '良心话',
 '获取',
 '几',
 '强迫',
 '日本',
 '玩越',
 '64',
 '下芬',
 '抽到',
 '涂色',
 '半天',
 '死不了',
 '师父',
 '好太多',
 '闪退点',
 '捣乱',
 '信用卡',
 '想开',
 '关求',
 '大赛',
 '换装备',
 '跨服',
 '残粉',
 '先去',
 '真烦',
 '帅哥美女',
 '爪',
 '集结',
 '祈祷',
 '空调',
 '拉圾',
 '厨房',
 '几条',
 '头发',
 '魔幻',
 '一分钱',
 '洛天',
 '来袭'

In [19]:
# model
# model.doesnt_match(["敌人","团队","浪漫"])
# model.similarity('识别', '枯燥')

In [22]:
# 根据 关键词 来组成词典, 并构成词向量，利用计算 向量间 夹角余弦值
from gensim import corpora
from gensim.matutils import corpus2csc
def doc2vec(df):
    # 生成词典
    tfidf_words = [x for x in df['tfidf_word']]
    tfidf_dict = corpora.Dictionary(tfidf_words)
    
    # doc2bow  生成词频向量， 和词频矩阵不同
    tfidf_word_count = [tfidf_dict.doc2bow(word) for word in tfidf_words] 
    # 词频矩阵
    # corpus2dense 稠密矩阵 csc 稀疏矩阵
    tfidf_matrix = corpus2csc(tfidf_word_count, len(tfidf_dict))
    return tfidf_matrix.T

In [23]:
tfidf_matrix = doc2vec(df)
type(tfidf_matrix)

scipy.sparse.csr.csr_matrix

In [24]:
tfidf_matrix.shape

(149909, 52024)

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
x = tfidf_matrix[:100]
# similarities = cosine_similarity(x)
# print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]



  (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 25)	1.0
  (0, 26)	1.0
  (0, 27)	1.0
  (0, 28)	1.0
  (0, 29)	1.0
  (0, 30)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 33)	1.0
  (0, 34)	1.0
  (0, 22)	1.0
  (0, 35)	1.0
  (0, 36)	1.0
  (0, 37)	1.0
  (0, 38)	1.0
  (0, 30)	1.0
  (0, 39)	1.0
  (0, 40)	1.0
  (0, 41)	1.0
  (0, 42)	1.0
  (0, 43)	1.0
  (0, 22)	1.0
  (0, 30)	1.0
  (0, 35)	1.0
  (0, 37)	1.0
  (0, 44)	1.0
  (0, 45)	1.0
  (0, 46)	1.0
  (0, 47)	1.0
  (0, 48)	1.0
  (0, 49)	1.0
  (0, 50)	1.0
  (0, 51)	1.0
  (0, 52)	1.0
  (0, 53)	1.0
  (0, 54)	1.0
  (0, 22)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 33)	1.0
  (0, 46)	1.0
  (0, 55)	1.0
  (0, 56)	1.0
  (0, 57)	1.0
  (0, 58)	1.0
  (0, 59)	1.0
  